In [ ]:
from value import Value
import random

In [26]:
class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.tanh()

    def parameters(self):
        return self.w + [self.b]

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

In [27]:
x = [2.0, 3.0, -1.0]
n = MLP(3,[4,4,1])

n(x)

Value(data = 0.5124633146585681)

In [28]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]

ys = [1.0, -1.0, -1.0, 1.0]

In [47]:
for k in range(20):

    #forward pass
    ypred = [n(x) for x in xs]
    loss = sum([(ygt - yout)**2 for ygt, yout in zip(ys,ypred)])

    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()

    for p in n.parameters():
        p.data += -0.01 * p.grad

    print(k, loss.data)


0 0.009652617726582956
1 0.009610243778574161
2 0.009568224785313316
3 0.009526556387554234
4 0.0094852342963636
5 0.00944425429171956
6 0.009403612221143567
7 0.009363303998364296
8 0.009323325602013103
9 0.009283673074350124
10 0.009244342520019804
11 0.009205330104835822
12 0.00916663205459362
13 0.009128244653910796
14 0.009090164245093872
15 0.009052387227031214
16 0.009014910054110955
17 0.008977729235163736
18 0.008940841332429464
19 0.008904242960547018


In [49]:
ypred

[Value(data = 0.9605807179615378),
 Value(data = -0.9559643875304802),
 Value(data = -0.9497848922473987),
 Value(data = 0.9462443402798543)]